In [22]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import logging as log

log.basicConfig(level=log.DEBUG)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HE

In [17]:
import sys
sys.path.append('../')
from baselines.utils import *
import os

In [29]:
train_data = load_data(os.path.join(cfg['DIR'], cfg['train_path']))

In [37]:
hypothesis = get_hypothesis(train_data)

In [35]:
train_data['documents'][0]['text'][535]

'('

In [25]:
log.debug(hypothesis)

DEBUG:root:{'nda-11': "receiving party shall not reverse engineer any objects which embody disclosing party's confidential information.", 'nda-16': 'receiving party shall destroy or return some confidential information upon the termination of agreement.', 'nda-15': 'agreement shall not grant receiving party any right to confidential information.', 'nda-10': 'receiving party shall not disclose the fact that agreement was agreed or negotiated.', 'nda-2': 'confidential information shall only include technical information.', 'nda-1': 'all confidential information shall be expressly identified by the disclosing party.', 'nda-19': 'some obligations of agreement may survive termination of agreement.', 'nda-12': 'receiving party may independently develop information similar to confidential information.', 'nda-20': 'receiving party may retain some confidential information even after the return or destruction of confidential information.', 'nda-3': 'confidential information may include verbally 

In [65]:
from icecream import ic

ModuleNotFoundError: No module named 'icecream'

In [69]:
from torch.utils.data import Dataset

class NLIDataset(Dataset):
    def __init__(self, data, tokenizer, hypothesis, max_len = 475):
        self.data = data
        self.tokenizer = tokenizer
        self.hypothesis = hypothesis
        self.max_len = max_len
        self.spans = []

        for i, doc in enumerate(self.data['documents']):
            for j, span in enumerate(doc['spans']):
                self.spans.append({
                    'doc_id': i,
                    'span_id': j,
                    'text': doc['text'][span[0]:span[1]]
                })

        self.data_points = []
        self.y = []

        print(self.spans[0]['span_id'])

        for key, val in self.hypothesis.items():
            for span in self.spans:
                self.y.append(int(span['span_id'] in self.data['documents'][span['doc_id']]['annotation_sets'][0]['annotations'][key]['spans']))
                self.data_points.append({
                    'hypotheis': val,
                    'premise': span['text']
                })

        self.tokenized_data = self.tokenizer(
            [data_point['hypotheis'] for data_point in self.data_points],
            [data_point['premise'] for data_point in self.data_points],
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
        
    def __len__(self):
        return len(self.data)

    # def __getitem__(self, index):

In [70]:
# text = "I love Adelaide!"
# # add_special_tokens=True is set by default
# text_enc = tokenizer.encode(text, text, padding='max_length', return_tensors='pt')

# log.debug(text_enc.shape)

# for tok in text_enc:
#     print(tok, tokenizer.decode(tok))

In [71]:
nliDataset = NLIDataset(train_data, tokenizer, hypothesis)

0
